# Model Management
## Azure ML and MLFlow tracking

In [ ]:
# Connect to workspace
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
import mlflow

credential = DefaultAzureCredential()
aml_client = MLClient(
    credential=credential,
    subscription_id="...",
    resource_group_name="...",
    workspace_name="...",
)
mlf_client = mlflow.tracking.MlflowClient()

In [ ]:
model_name = 'sklearn.ensemble._forest.RandomForestRegressor'

models = [x 
    for x in aml_client.models.list() 
    if x.name==model_name
]

for i,m in enumerate(models):
    print(i, m.name, m.latest_version)

## How to pick the right model among models?

In [ ]:
# for model in client.search_registered_models():
#     print(f"{model.name}")

mlf_client.search_registered_models(f"name='{model_name}'")
models = mlf_client.search_model_versions(f"name='{model_name}'")

# m3 = mlf_client.get_model_version(model_name, 3) 
m2 = mlf_client.get_model_version(model_name, 5)
m1 = mlf_client.get_model_version(model_name, 4)

def get_rmse(m):
    run = mlf_client.get_run(m.run_id)
    r = run.data.metrics['rmse']
    return r

candidate_model = None
if get_rmse(m1)<get_rmse(m2):
    # deploy m1
    candidate_model = m1
else:
    # deploy m2
    candidate_model = m2

print(f'Deploy model "{candidate_model.name}" version "{candidate_model.version}" ')

In [ ]:
# find the model currently in production
endpoint_name = 'my-endpoint2'
deployment_name = 'blue2'


from azure.ai.ml.entities import Model

def get_model_from_uri(uri):
    tokens = uri.split('/')
    tid = tokens.index('models')
    model = tokens[tid+1]
    vid = tokens.index('versions')
    version = tokens[vid+1]
    return aml_client.models.get(model, version)

def get_deployed_model(endpoint_name, deployment_name)->Model:
    endpoint = aml_client.online_endpoints.get(endpoint_name)
    deployment = aml_client.online_deployments.get(deployment_name, endpoint.name)
    deployed_model = get_model_from_uri(deployment.model)
    return deployed_model

def get_model_run(model: Model):
    client = mlflow.tracking.MlflowClient()
    return client.get_run(model.job_name)

def compare_models(old_model, new_model, metrics):
    old_run = get_model_run(old_model)
    new_run = get_model_run(new_model)
    for metric in metrics:
        print(f'- {metric}: {old_run.data.metrics.get(metric)}  vs {new_run.data.metrics.get(metric)}')

deployed_model = get_deployed_model(endpoint_name, deployment_name)


new_model = aml_client.models.get(candidate_model.name, candidate_model.version)

compare_models( deployed_model, new_model, [
    'training_mean_squared_error',
    'training_mean_absolute_error', 
    'training_r2_score', 
    'training_root_mean_squared_error', 
    'training_score','rmse'
])